# Anexo 1. Preparación de dataset Openflights

Preparación de dataset de vuelos áreos publicada por el sitio web [Openflights](https://openflights.org/data.html). La base de datos contiene una tabla de los aeropuertos en el mundo (**`aeropuertos.csv`**), una tabla de las aerolíneas en el mundo (**`aerolineas.csv`**) y una tabla de las rutas de vuelos comerciales (**`rutas.csv`**).

Más información en https://github.com/jpatokal/openflights

In [ ]:
aeropuertos_file='data/airports.dat'
aeropuertos=pd.read_csv(aeropuertos_file, header=None)

aeropuertos.head()

In [ ]:
columnas=['Id','Nombre','Ciudad','Pais','IATA','OACI','Latitud','Longitud','Altitud','Zona_horaria','Horario_verano','Zona_horaria_lugar','Tipo','Fuente']
aeropuertos.columns = columnas
aeropuertos = aeropuertos.replace("\\N", value=np.nan)
aeropuertos['Altitud']=np.round(aeropuertos['Altitud']*0.3048,1)
aeropuertos[['Zona_horaria','Altitud']] = aeropuertos[['Zona_horaria','Altitud']].apply(pd.to_numeric)
aeropuertos.head()

In [ ]:
aeropuertos.dtypes

In [ ]:
aeropuertos.to_csv('data/aeropuertos.csv', index=False)

In [ ]:
rutas_file='data/routes.dat'
rutas=pd.read_csv(rutas_file, header=None)

rutas.head()

In [ ]:
columnas=['Linea_aerea','Linea_aerea_Id','Aeropuerto_origen','Aeropuerto_origen_Id','Aeropuerto_destino','Aeropuerto_destino_Id','Vuelo_operado','Numero_paradas','Modelo_avion']
rutas.columns = columnas
rutas = rutas[(rutas["Linea_aerea_Id"] != "\\N") & (rutas["Aeropuerto_origen_Id"] != "\\N") & (rutas["Aeropuerto_destino_Id"] != "\\N")]
rutas[['Linea_aerea_Id','Aeropuerto_origen_Id','Aeropuerto_destino_Id']] = rutas[['Linea_aerea_Id','Aeropuerto_origen_Id','Aeropuerto_destino_Id']].apply(pd.to_numeric)
rutas.head()

In [ ]:
rutas.dtypes

In [ ]:
def pais_aeropuerto(row, name):
    result='Unknown'
    try:
        result = aeropuertos[aeropuertos["Id"] == row[name]].iloc[0]
        result = result['Pais']
    except (ValueError, IndexError):
        pass
    return result

In [ ]:
rutas['Pais_origen']=rutas.apply(pais_aeropuerto, name='Aeropuerto_origen_Id', axis=1)
rutas['Pais_destino']=rutas.apply(pais_aeropuerto, name='Aeropuerto_destino_Id', axis=1)

In [ ]:
import math

def haversine(lon1, lat1, lon2, lat2):
    # Convert coordinates to floats.
    lon1, lat1, lon2, lat2 = [float(lon1), float(lat1), float(lon2), float(lat2)]
    # Convert to radians from degrees.
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    # Compute distance.
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    km = 6367 * c
    return km

def calcular_distancia(row):
    dist = 0
    try:
        # Match source and destination to get coordinates.
        source = aeropuertos[aeropuertos["Id"] == row["Aeropuerto_origen_Id"]].iloc[0]
        dest = aeropuertos[aeropuertos["Id"] == row["Aeropuerto_destino_Id"]].iloc[0]
        # Use coordinates to compute distance.
        dist = haversine(dest["Longitud"], dest["Latitud"], source["Longitud"], source["Latitud"])
    except (ValueError, IndexError):
        pass
    return dist

In [ ]:
rutas['Distancia_km'] = rutas.apply(calcular_distancia, axis=1)

In [ ]:
rutas.to_csv('data/rutas.csv', index=False)

In [ ]:
aerolineas_file='data/airlines.dat'
aerolineas=pd.read_csv(aerolineas_file, header=None)

aerolineas.head()

In [ ]:
columnas=['Linea_aerea_Id','Linea_aerea','Alias','IATA','OACI','Sigla_identificacion','Pais','Activa']
aerolineas.columns = columnas
aerolineas = aerolineas.replace("\\N", value=np.nan)
aerolineas['Linea_aerea_Id'] = aerolineas['Linea_aerea_Id'].apply(pd.to_numeric)
aerolineas.head()

In [ ]:
aerolineas.dtypes

In [ ]:
aerolineas.to_csv('data/aerolineas.csv', index=False)